In [18]:
import pandas as pd
import gzip
import ast
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import scipy.stats as stats 


In [19]:
from funcion_desanidar import descomprimirjson

user_items = descomprimirjson("./Datasets/users_items.json.gz",'items')

Visualizacion de los datos del dataframe

Con el Análisis Exploratorio de Datos (EDA) podremos comprender los datos, identificar patrones y hacer observaciones iniciales.

In [20]:
user_items.info()# Mostramos el contenido del dataframe (nºregistros, etiquetas de cabeceras, tipo de datos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


In [22]:
user_items.head() #mostrar las primeras filas del dataframe

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0


In [25]:
user_items.nunique()  # Recuperamos los valores únicos de cada columna

user_id             87626
items_count           925
steam_id            87625
user_url            87626
item_id             10978
item_name           10947
playtime_forever    48861
playtime_2weeks      4450
dtype: int64

In [26]:
user_items.describe(include='all') # resumen estadístico de las variables numéricas, include='all' proporciona un resumen adicional de los datos

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
count,5170015,5.170015e+06,5170015,5170015,5153209,5153209,5.153209e+06,5.153209e+06
unique,87626,NaN,87625,87626,10978,10947,NaN,NaN
top,phrostb,NaN,76561198102767019,http://steamcommunity.com/id/phrostb,205790,Dota 2 Test,NaN,NaN
freq,7762,NaN,7762,7762,49571,49571,NaN,NaN
mean,NaN,3.136999e+02,NaN,NaN,NaN,NaN,9.914951e+02,9.104707e+00
std,NaN,6.924635e+02,NaN,NaN,NaN,NaN,5.418204e+03,1.403926e+02
min,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
25%,NaN,7.100000e+01,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
50%,NaN,1.370000e+02,NaN,NaN,NaN,NaN,3.400000e+01,0.000000e+00
75%,NaN,2.750000e+02,NaN,NaN,NaN,NaN,3.550000e+02,0.000000e+00


Valores duplicados

In [33]:
user_items.duplicated().sum() #para contar si hay valores duplicados

59196

In [34]:
user_items.drop_duplicates(inplace=True)  #eliminar los valores duplicados. El parámetro inplace nos permite indicar si queremos que los valores duplicados se eliminen directamente en el DataFrame

Manejo de valores faltantes

Vamos a evaluar el total de valosres faltantes en cada columna y que porcentaje del dataframe representan para poder evaluar si los eliminamos o completamos los datos faltantes.

In [39]:
total = user_items.isnull().sum().sort_values(ascending = False)
porcentaje = (user_items.isnull().sum() / user_items.count()).sort_values(ascending = False)
datos_faltantes = pd.concat([total, porcentaje], axis = 1, keys = ['Total', 'Porcentaje'])
datos_faltantes

,Total,Porcentaje
item_id,16714,0.003281
item_name,16714,0.003281
playtime_forever,16714,0.003281
playtime_2weeks,16714,0.003281
user_id,0,0.000000
items_count,0,0.000000
steam_id,0,0.000000
user_url,0,0.000000


Dado que las horas jugadas son un dato que necesitamos para utilizar en una de nuestras funciones, y los datos nulos no nos proporcionan ninguna informacion procefemos a eliminarlos

In [43]:
user_items.dropna(inplace=True)

In [44]:
total = user_items.isnull().sum().sort_values(ascending = False)
porcentaje = (user_items.isnull().sum() / user_items.count()).sort_values(ascending = False)
datos_faltantes = pd.concat([total, porcentaje], axis = 1, keys = ['Total', 'Porcentaje'])
datos_faltantes

,Total,Porcentaje
user_id,0,0.0
items_count,0,0.0
steam_id,0,0.0
user_url,0,0.0
item_id,0,0.0
item_name,0,0.0
playtime_forever,0,0.0
playtime_2weeks,0,0.0


La visualización grafica de datos permite descubrir de una manera simple patrones y tendencias, ya que a través de los graficoa los datos pueden ser analizados y entendidos de manera intuitiva y fácil de interpretar.

In [32]:
variables = ['items_count', 'playtime_forever', 'playtime_2weeks']
columns = 4
fig, axes = plt.subplots(len(variables) //columns, columns, figsize=(15,8))
for current_idx, variable in enumerate(variables):
    i = current_idx // columns
    j = current_idx % columns
    sns.distplot(user_items[variable], ax=axes[i][j])
    axes[i][j].set_title(variable)
    axes[i][j].set_xlabel("")
plt.tight_layout()


ValueError: Number of rows must be a positive integer, not 0

<Figure size 1500x800 with 0 Axes>